In [ ]:
from google.colab import files
import os

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [ ]:
model_path = "/content/drive/MyDrive/yolov8s.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [ ]:
import yaml


# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

In [ ]:
import os
from ultralytics import YOLO
import numpy as np


def train_yolov8(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    """
    Train YOLOv8 model with enhanced settings for small overlapping object detection.
    """
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # Train the model with optimized settings
    print(f"Training YOLOv8 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolov8_training",
        iou=iou_threshold,  # Adjust IoU threshold for better object separation
        mosaic=mosaic,  # Enhance data augmentation with mosaic images
        mixup=mixup,  # Mixup augmentation for diverse training
        degrees=180,  # Rotation augmentation
    )

    print(f"Training complete. Results saved to {output_dir}")

# Colab 用にパラメータを手動で指定
model_path = "/content/drive/MyDrive/yolov8s.pt"  # モデルのパス (Google Drive に保存している場合は適宜変更)
#data_yaml = "/content/drive/MyDrive/MaiNDLab/dataset/data.yaml"  # データセットの YAML ファイル
epochs = 200  # エポック数
batch_size = 64  # バッチサイズ
img_size = 960  # 画像サイズ
iou_threshold = 0.3  # IoU 閾値
mosaic = 1.0  # モザイクの確率
mixup = 0.2  # Mixup の確率
output_dir = "runs_experiment"  # 出力ディレクトリ

# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLOv8 の学習実行
train_yolov8(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)


In [ ]:
# best.pt のパス
best_model_path = 'runs_experiment/yolov8_training/weights/best.pt'

# best.pt が存在すれば自動でダウンロード
if os.path.exists(best_model_path):
    files.download(best_model_path)
else:
    print("best.pt が見つかりませんでした。")